#### Пример использования дедубликатора

In [1]:
%pip install faiss-cpu --no-cache

     ---------------------------------------- 10.8/10.8 MB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Препроцессим столбец "text" из датафрейма с новостями

In [1]:
# т.к. относительный импорт не работает необходимо подсоединить к 
# изучаемой дериктории родительскую дерикторию
import os
import sys
import pandas as pd

sys.path.append(os.path.dirname(os.getcwd()))

# препроцессим столбец с текстом
from utils import preprocessing

samples = pd.read_excel('../utils/posts.xlsx')[['title', 'text']]
preprocesserClass = preprocessing.textPreprocesser(samples.copy(),['text'])
preprocesserClass.clean()
preprocesserClass.df.head()

,title,text
0,Восстановление аммиакопровода займет до 3 меся...,"[восстановлен, аммиакопровод, тольятт, одесс, ..."
1,Выделены дополнительные 39 млн рублей на соцуч...,"[дополнительн, миллион, рубл, был, выдел, на, ..."
2,«Россия уничтожена санкциями»\n ...,"[росс, уничтож, санкц, байд, ., ах, ха, ха]"
3,Обыски в минобразования Дагестана по делу о вы...,"[правоохранител, провел, обыск, в, министерств..."
4,"Реконструкция пропускного пункта ""Верхний Ларс...","[реконструкц, пропускн, пункт, верхн, ларс, за..."


In [ ]:
preprocesserClass.df.shape
preprocesserClass.df.drop_duplicates(subset='title', inplace=True)
preprocesserClass.df.shape

Создаем простейшие эмбеддинги на основе алгоритма TF-IDF

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = preprocesserClass.df['text'].str.join(' ')
vectorizer = TfidfVectorizer()
embeddings = vectorizer.fit_transform(corpus)
embeddings.shape

(169300, 145419)

Находим расстояния между эмбеддингами

In [3]:
from utils import deduplicator
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix

embeddings = csr_matrix(embeddings).astype(np.float32).toarray().astype(np.float16)
dists, indices = deduplicator.l2_dists(embeddings)

MemoryError: Unable to allocate 45.9 GiB for an array with shape (169300, 145419) and data type float16

На основе полученных расстояний удаляем дубликаты из датафрейма с новостями

In [ ]:
dedup_df, removed_n = deduplicator.dedup(preprocesserClass.df, dists, indices)
print(f'Процент удаленных дубликатов: {round(removed_n / preprocesserClass.df.shape[0] * 100, 2)}')

Процент удаленных дубликатов: 44.19


In [ ]:
dedup_df.head()

,title,text
23298,**🇩🇪 Немка пожаловалась на домогательства со с...,"[немк, пожалова, на, домогательств, со, сторон..."
154293,Минувшей ночью в небе над городом,"[минувш, ноч, в, неб, над, город]"
97458,**Підведені підсумки місяця пілотного проекту ...,"[п, двед, п, дсумк, м, сят^, п, лотн, проект, ..."
100288,"Скажите, а Вы тоже, уходя из дома, говорите ко...","[скаж, а, вы, уход, из, дом, говор, кот, ил, с..."
80350,**Инвесторы вывели с начала 2023 года $27 млрд...,"[инвестор, вывел, с, нача, год, млрд, из, евро..."
